Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

In [7]:
#@title  Añadir soporte GGUF a ComfyUI
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF.git
%cd $WORKSPACE/custom_nodes/ComfyUI-GGUF
!pip install -r requirements.txt



/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-GGUF' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes/ComfyUI-GGUF
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 9.4 MB/s eta 0:00:00


In [ ]:
#@title descargamos un modelo guff
%cd $WORKSPACE/models/unet
!wget -nc "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q3_K_S.gguf"

In [16]:
#@title Descarga de modelos (CLIP) requeridos para DualCLIPLoader (GGUF) con nombres correctos
%cd $WORKSPACE

!mkdir -p models/clip
%cd models/clip

# Descargar CLIP y renombrar a clip-vit-large-patch14.bin
!wget -nc -O CLIP-GmP-ViT-L-14.safetensors "https://huggingface.co/zer0int/CLIP-GmP-ViT-L-14/resolve/main/model.safetensors?download=true"

# Descargar T5 y renombrar a model.safetensors
!wget -nc -O t5-v1_1-xxl-encoder-bf16.safetensors "https://huggingface.co/city96/t5-v1_1-xxl-encoder-bf16/resolve/main/model.safetensors?download=true"

# Verifica los archivos
!ls -lh $WORKSPACE/models/clip/


/content/ComfyUI
/content/ComfyUI/models/clip
--2025-05-13 18:43:35--  https://huggingface.co/zer0int/CLIP-GmP-ViT-L-14/resolve/main/model.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.34, 13.35.202.121, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/35/a0/35a091c7ee1046ffdee4a1498b37c36845fab5f28772cd5245788944c2360166/0b820c4ef9a06ed7a5872b064405a69e26ebba5f524c92263b9bec1692a71bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747165415&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzE2NTQxNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM1L2EwLzM1YTA5MWM3ZWUxMDQ2ZmZkZWU0YTE0OThiMzdjMzY4NDVmYWI1ZjI4NzcyY2Q1MjQ1Nzg4OTQ0YzIzNjAxNjYvMGI4MjBjNGVmOWEwNmVkN2E1ODcyYjA2N

In [13]:
#@title descargamos el vae
%cd $WORKSPACE/models/vae
!wget https://huggingface.co/StableDiffusionVN/Flux/resolve/main/Vae/flux_vae.safetensors

/content/ComfyUI/models/vae
--2025-05-13 18:33:38--  https://huggingface.co/StableDiffusionVN/Flux/resolve/main/Vae/flux_vae.safetensors
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/b8/a5/b8a57ba4d3c6088d6f4a9b539cdb398c3b22dd6fb56a7b4df77bf2f4911f4442/afc8e28272cd15db3919bacdb6918ce9c1ed22e96cb12c4d5ed0fba823529e38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux_vae.safetensors%3B+filename%3D%22flux_vae.safetensors%22%3B&Expires=1747164818&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzE2NDgxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2I4L2E1L2I4YTU3YmE0ZDNjNjA4OGQ2ZjRhOWI1MzljZGIzOThjM2IyMmRkNmZiNTZhN2I0ZGY3N2JmMmY0OTExZjQ0NDIvYWZjOGUyODI3MmNkMTVkYjM5MTliYWNkYjY5MThjZTljMWVkMjJlOTZjYjE

In [ ]:
#@title Run ComfyUI with cloudflared (Recommended Way)
%cd $WORKSPACE
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

/content/ComfyUI
--2025-05-13 19:28:26--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.2/cloudflared-linux-amd64.deb [following]
--2025-05-13 19:28:26--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.2/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/f6f8a9db-3887-4a1c-9ecb-9781f97395a5?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250513%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250513T192826Z&X-Amz-Expires=300&X-Amz-Signature=8f75593da5e9e217a16fb9d5b680ffff32d0aab